### Loading Word2Vec Data

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-11-17 18:29:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-17 18:29:54--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-17 18:29:55--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
!unzip glove.6B.zip


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
import numpy as np
embeddings_index = {}
f = open( 'glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


### PreProcessing

In [4]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from keras.layers import  Flatten, Dense, Dropout


Using TensorFlow backend.


In [20]:
test=pd.read_csv("./test.csv")
train=pd.read_csv("./train.csv")
print(train.head(5))


     ID                                               Text  Class
0  7303  "Please enable removing language code from the...      1
1  5258  "Note: in your .csproj file, there is a Suppor...      0
2  6218  "Wich means the new version not fully replaced...      0
3  3076  "Some of my users will still receive the old x...      0
4  6508  "The store randomly gives the old xap or the n...      0


In [21]:
x_train=train['Text']
y_train=train[['Class']]
train.info


<bound method DataFrame.info of         ID                                               Text  Class
0     7303  "Please enable removing language code from the...      1
1     5258  "Note: in your .csproj file, there is a Suppor...      0
2     6218  "Wich means the new version not fully replaced...      0
3     3076  "Some of my users will still receive the old x...      0
4     6508  "The store randomly gives the old xap or the n...      0
...    ...                                                ...    ...
8495  5149  The following will reliably throw some flavor ...      0
8496  3794  This will reliably result in a BSOD if zero by...      0
8497  5384  As of now it's only possible to set the Calend...      0
8498   859  It's not possible to set the Hijri adjustment ...      0
8499  6659  The Windows 10 Calender app show cases this fe...      0

[8500 rows x 3 columns]>

In [22]:
test.info

<bound method DataFrame.info of         ID                                               Text
0     9348         This would enable live traffic aware apps.
1     9393  Please try other formatting like bold italics ...
2     8557  Since computers were invented to save time I s...
3     9288  Allow rearranging if the user wants to change ...
4     9231  Add SIMD instructions for better use of ARM NE...
...    ...                                                ...
1420  8583  we should have small tiles instead of a long l...
1421  9409  An app should be able to publish a service tha...
1422  8733  For example, if I have an app that can process...
1423  9697  I would like access to a stream for music play...
1424  9798  i wish the functionality to access and synchro...

[1425 rows x 2 columns]>

In [23]:

y_train=np.array(y_train).reshape((-1))
print(y_train)

[1 0 0 ... 0 0 0]


In [24]:
np.average(y_train)

0.24529411764705883

In [0]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [26]:
x_train.values

array(['"Please enable removing language code from the Dev Center "language history" For example if you ever selected "ru" and "ru-ru" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad."',
       '"Note: in your .csproj file, there is a SupportedCultures entry like this: <SupportedCultures>de-DE;ru;ru-RU </SupportedCultures> When I removed the "ru" language code and published my new xap version, the old xap version still remains in the Store with "Replaced and unpublished"."',
       '"Wich means the new version not fully replaced the old version and this causes me very serious problems: 1."',
       ...,
       "As of now it's only possible to set the CalenderIdentifier property to tell the control which calender to use.",
       "It's not possible to set the Hijri adjustment for the month of Ramadan/Shaban though which can vary per region/user (some users like to use the adjustment of a differen

In [27]:
word_index=[]
num_words = 10000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ')
tokenizer.fit_on_texts(x_train.values)
X = tokenizer.texts_to_sequences(x_train.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 200
X = pad_sequences(X, maxlen=max_length_of_text)

print(word_index)


Found 10001 unique tokens.
{'the': 1, 'to': 2, 'a': 3, 'and': 4, 'in': 5, 'is': 6, 'i': 7, 'it': 8, 'of': 9, 'for': 10, 'be': 11, 'that': 12, 'this': 13, 'app': 14, 'on': 15, 'with': 16, 'not': 17, 'have': 18, 'windows': 19, 'would': 20, 'you': 21, 'as': 22, 'an': 23, 'if': 24, 'or': 25, 'but': 26, 'can': 27, 'when': 28, 'my': 29, 'apps': 30, 'like': 31, 'are': 32, 'from': 33, 'phone': 34, 'we': 35, 'so': 36, 'more': 37, 'use': 38, '\r': 39, 'there': 40, 'should': 41, 'all': 42, 'will': 43, 'user': 44, 'which': 45, 'by': 46, 'please': 47, 'add': 48, 'only': 49, 'store': 50, 'at': 51, 'could': 52, 'using': 53, 'one': 54, 'do': 55, 'microsoft': 56, 'no': 57, '1': 58, 'support': 59, 'some': 60, 'way': 61, 'new': 62, 'allow': 63, 'make': 64, 'com': 65, 'has': 66, 'able': 67, 'uwp': 68, 'also': 69, 'application': 70, 'currently': 71, 'just': 72, 'control': 73, 'feedly': 74, 'get': 75, 'want': 76, 'was': 77, 'then': 78, 'other': 79, 'api': 80, 'need': 81, 'time': 82, 'they': 83, '0': 84, 'an

In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
X_train,X_test, y_train, y_test = train_test_split(X,y_train, test_size = 0.05, random_state = 42)
# print(X_train.shape,y_train.shape)
# X_train
# print(X_test.shape,y_test.shape)

In [30]:
X_test.shape

(425, 200)

### Creating Model

In [54]:
import keras
from keras.layers import Convolution1D as Conv1D
from  keras.layers import MaxPooling1D as MaxPooling1D

embed_dim = 200 #Change to observe effects
lstm_out = 512 #Change to observe effects
batch_size = 32


input1 = keras.layers.Input(shape=(200,))

x = Embedding(10002, embed_dim,weights=[embedding_matrix])(input1)  
x = Conv1D(512, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)

x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)  # global max pooling
x = Flatten()(x)
print(x.shape)
x = Dense(128, activation='relu')(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model(input1, preds)
print(model.summary())
import keras

# input1 = keras.layers.Input(shape=(16,))
# x1 = keras.layers.Dense(8, activation='relu')(input1)
# input2 = keras.layers.Input(shape=(32,))
# x2 = keras.layers.Dense(8, activation='relu')(input2)
# # equivalent to added = keras.layers.add([x1, x2])
# added = keras.layers.Add()([x1, x2])

# out = keras.layers.Dense(4)(added)
# model = keras.models.Model(inputs=[input1, input2], outputs=out)


(?, ?)
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 200, 200)          2000400   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 196, 512)          512512    
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 192, 256)          655616    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 38, 256)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 34, 256)           327936    
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 30, 128)        

In [0]:
# embed_dim = 128
# lstm_out = 196
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

# model = Sequential()
# model.add(Embedding(2000, embed_dim,input_length = X_train.shape[1]))
# model.add(SpatialDropout1D(0.3))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(512,activation='relu'))
# model.add(Dense(128,activation='relu'))
# model.add(Dense(64,activation='relu'))
# model.add(Dense(10,activation='softmax'))

# # model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())

In [0]:
import keras
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [58]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# X_train,X_test, X_train_class, X_test_class, y_train, y_test =
model.fit(x=[X_train], y=y_train, batch_size=100, epochs=8,callbacks=callbacks_list, verbose=1,validation_data=([X_test],y_test),class_weight= {0:1,1:3})

Train on 8075 samples, validate on 425 samples
Epoch 1/8
8075/8075 [==============================] - 9s 1ms/step - loss: 0.8403 - acc: 0.6996 - val_loss: 0.3677 - val_acc: 0.8894

Epoch 00001: val_acc improved from -inf to 0.88941, saving model to weights-improvement.hdf5
Epoch 2/8
8075/8075 [==============================] - 7s 890us/step - loss: 0.5358 - acc: 0.8510 - val_loss: 0.3665 - val_acc: 0.8659

Epoch 00002: val_acc did not improve from 0.88941
Epoch 3/8
8075/8075 [==============================] - 7s 885us/step - loss: 0.3864 - acc: 0.8941 - val_loss: 0.2969 - val_acc: 0.8894

Epoch 00003: val_acc improved from 0.88941 to 0.88941, saving model to weights-improvement.hdf5
Epoch 4/8
8075/8075 [==============================] - 7s 894us/step - loss: 0.2856 - acc: 0.9226 - val_loss: 0.3422 - val_acc: 0.8753

Epoch 00004: val_acc did not improve from 0.88941
Epoch 5/8
8075/8075 [==============================] - 7s 892us/step - loss: 0.2101 - acc: 0.9516 - val_loss: 0.3335 - val

In [0]:
Y=tokenizer.texts_to_sequences(test['Text'].values)


testing_input = pad_sequences(Y, maxlen=max_length_of_text)


In [0]:
model.load_weights("weights-improvement.hdf5")

In [0]:
predict=model.predict([testing_input])

In [62]:
print(predict[0:50])

[[0.06487566]
 [0.92742074]
 [0.8962215 ]
 [0.9716372 ]
 [0.916628  ]
 [0.5963221 ]
 [0.91474485]
 [0.8623058 ]
 [0.7037799 ]
 [0.99176997]
 [0.91463184]
 [0.9234885 ]
 [0.99008954]
 [0.56387305]
 [0.97391677]
 [0.9758331 ]
 [0.9560262 ]
 [0.0670805 ]
 [0.44240755]
 [0.96319425]
 [0.02692831]
 [0.937729  ]
 [0.86354506]
 [0.01504403]
 [0.04906622]
 [0.99029875]
 [0.5713516 ]
 [0.8511274 ]
 [0.99631214]
 [0.07691628]
 [0.83048564]
 [0.02593523]
 [0.39140117]
 [0.943929  ]
 [0.06730264]
 [0.03902078]
 [0.7741159 ]
 [0.6641135 ]
 [0.39668465]
 [0.06953809]
 [0.03221014]
 [0.6831061 ]
 [0.989115  ]
 [0.08116323]
 [0.93918675]
 [0.9935678 ]
 [0.94193065]
 [0.44040483]
 [0.9446311 ]
 [0.8925314 ]]


In [0]:
id=np.array(test['ID'])



### Creating Submission files

In [0]:

for i in range(len(id)):
    if predict[i]>0.5:
        predict[i]=1
    else:
        predict[i]=0
predict=predict.reshape(-1)
import csv
sub=open("sub1.csv","w")
writer=csv.writer(sub)

writer.writerow(['ID','Class'])
for i in range(len(id)):
    writer.writerow([id[i],int(predict[i])])
sub.close()


<a href="sub1.csv">subA</a>

* <a href="/content/weights-improvement.hdf5">download</a>

In [0]:
# predictB=predictB
predictB*=10
for i in range(53175):
    if predictB[i]>10:
        predictB[i]=10.0
    elif predictB[i]<1:
        predictB[i]=1.0
    else:
        pass
temp=np.ceil(predictB)
temp.reshape(53175)
import csv
sub=open("sub2.csv","w")
writer=csv.writer(sub)
writer.writerow(["Id","rating"])
for i in range(0,53175):
    writer.writerow([i,temp[i]])
sub.close()


<a href="sub2.csv">subB</a>

In [0]:
predictC=predictC/np.max(predictC)
predictC*=10
# predictC=predictC)
for i in range(53175):
    if predictC[i]>10:
        predictC[i]=10.0
    elif predictC[i]<1:
        predictC[i]=1.0
    else:
        pass
# y_test=np.ceil(predictB)
temp=predictC.reshape(-1)
import csv
sub=open("sub3.csv","w")
writer=csv.writer(sub)
writer.writerow(["Id","rating"])
for i in range(0,53175):
    writer.writerow([i,temp[i]])
sub.close()


<a href="sub3.csv">subC</a>

In [0]:
print('done')

done


In [0]:
model.save('weights-best.hdf5')

In [0]:
!ls

drive  sample_data  sub1.csv  test.csv	train.csv  weights-improvement.hdf5


In [0]:
!mv /content/weights-improvement.hdf5 /content/drive/My\ Drive/

* <a href="weights-best.hdf5">download</a>